In [0]:
import pandas
import geopy
from geopy.geocoders import Nominatim
import requests
from bs4 import BeautifulSoup
import numpy as np
from time import sleep

In [0]:
alphabet = ["A", "B", "C", "Č", "Ć", "D", "DŽ", "Đ", "E", "F", "G", "H", "I", "J", "K", "L", "LJ", "M", "N", "NJ", "O", "P", "R", "S", "Š", "T", "U", "V", "Z", "Ž"]

In [0]:
places = np.zeros((6790, 4), dtype=object)
index = 0
for letter in alphabet:
  result = requests.get("https://hr.wikipedia.org/wiki/Dodatak:Abecedni_popis_naselja_u_Republici_Hrvatskoj/{}".format(letter))
  src = result.content
  soup = BeautifulSoup(src, 'lxml')
  table = soup.find_all("a")

  flag = 0
  count = 1
  for i, row in enumerate(table):
    if "Dodatak:Popis gradova" in row.text:
      flag = 1
      continue

    if "↑" in row.text or "uredi VE" in row.text or "http" in row.text:
      flag = 0

    if "[" in row.text:
      continue

    if flag == 1:
      if count == 1 or count == 4:
        places[index, 0] = table[i].text
        places[index, 1] = table[i+1].text
        index += 1
        count = 1

      count += 1

In [4]:
print(places)

[['Ada' 'Šodolovci' 0 0]
 ['Adamovec' 'Zagreb' 0 0]
 ['Adžamovci' 'Rešetari' 0 0]
 ...
 ['Žutnica' 'Krapina' 0 0]
 ['Žužići' 'Višnjan' 0 0]
 ['Žužići' 'Tinjan' 0 0]]


In [5]:
print(places.shape)

(6790, 4)


In [0]:
places_notfound = []

In [7]:
#Fetching longitudes and latitudes
nom = Nominatim(user_agent="croatiaplaces2")

for i in range(places.shape[0]):
  n = nom.geocode(places[i, 0] + ", " + places[i, 1] + ", Croatia")
  if n == None:
    sleep(1.2)
    n = nom.geocode(places[i, 0] + ", Croatia")
    if n == None:
      places_notfound.append([places[i, 0], places[i, 1]])
      sleep(1.2)
      continue
  
  places[i, 2] = n.latitude
  places[i, 3] = n.longitude
  print("Broj: {}, Selo: {}".format(i, places[i, 0]))
  sleep(1.2)

Streaming output truncated to the last 5000 lines.
Broj: 1295, Selo: Draga Bašćanska
Broj: 1297, Selo: Draga Svetojanska
Broj: 1298, Selo: Dragalić
Broj: 1299, Selo: Draganić
Broj: 1300, Selo: Draganić
Broj: 1302, Selo: Draganovec
Broj: 1303, Selo: Draganje Selo
Broj: 1304, Selo: Drage
Broj: 1305, Selo: Drage
Broj: 1306, Selo: Dragljane
Broj: 1307, Selo: Dragonoš
Broj: 1308, Selo: Dragoslavec Breg
Broj: 1309, Selo: Dragoslavec Selo
Broj: 1310, Selo: Dragoslavec
Broj: 1311, Selo: Dragoševci
Broj: 1312, Selo: Dragošička
Broj: 1313, Selo: Dragotin
Broj: 1314, Selo: Dragotina
Broj: 1316, Selo: Dragovanščak
Broj: 1317, Selo: Dragovci
Broj: 1318, Selo: Dragove
Broj: 1319, Selo: Dragović
Broj: 1321, Selo: Dragozetići
Broj: 1322, Selo: Draguć
Broj: 1323, Selo: Draguzeti
Broj: 1324, Selo: Drakulić Rijeka
Broj: 1325, Selo: Dramalj
Broj: 1326, Selo: Drastin
Broj: 1327, Selo: Drašći Vrh
Broj: 1328, Selo: Draše
Broj: 1330, Selo: Draškovec
Broj: 1331, Selo: Drašković
Broj: 1332, Selo: Drašnice
Broj:

In [0]:
print(places[0:50])

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
np.savetxt("/content/drive/My Drive/Croatia Places/placesUpdate.txt", places, delimiter=",", fmt="%s")

In [11]:
notFound = np.array(places_notfound)
print(notFound.shape)

(617, 2)


In [0]:
np.savetxt("/content/drive/My Drive/Croatia Places/placesNotFoundUpdate.txt", notFound, delimiter=",", fmt="%s")